In [1]:
from pyautogui import *
import pyautogui
import time
import keyboard
import random
import win32api, win32con
import datetime as dt
from IPython.display import clear_output

In [2]:
my_courses={'EE442':'Microwave Engineering Lab',
'EE499':'BTP',
'EE626':'Pattern Recognition and Machine Learning',
'HS-401':'Management of Organizational Behaviour',
'EE674':'Sychrophasor Technology',
'EE672':'Power Electronics- Chandan Kumar',
'EE 661':'Power Electronics for Renewable Energy Systems',
'RFIC':'RF circuits: Mahima Arawati'}


In [3]:
teams_button=['teams_button.png','teams_button_active.png','teams_button_active.png']
search_bars=['search_bar.png','search_bar2','search_bar3']
join_button=['join.png']
video_off=['off_video_button.png']
select_audio=['computer_audio.png']
audio_off=['off_audio_button.png']
join_now=['join_now.png']
leave_call=['Leave_call.png','Leave_call_small.png']
meeting_now=['meeting_now.png']

In [4]:
def click(x,y):
    win32api.SetCursorPos((x,y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,0,0)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,0,0)

In [5]:
def click_button(button_images,try_this=50,gs=True):
    flag=1
    while (flag and try_this>0):
        try_this-=1
        for button_image in button_images:
            button_location = pyautogui.locateOnScreen(button_image, grayscale=gs, confidence=0.8)
            if (button_location != None):
                flag=0
                break
    if flag==1:
        return 0
    button_centre = pyautogui.center(button_location)
    pyautogui.click(button_centre.x, button_centre.y)
    return button_centre

In [6]:
def leave_call_func():
    click_button(teams_button,try_this=10)
    time.sleep(1)
    click_button(meeting_now,try_this=10)
    time.sleep(1)
    click_button(leave_call,try_this=10)

In [7]:
def time_diff(time1,time2):
    datetime1 = dt.datetime.combine(dt.date.today(), time1)
    datetime2 = dt.datetime.combine(dt.date.today(), time2)
    return datetime1 - datetime2

def attend_lec(course_name,attend_time_hrs,attend_time_mins,finish_time,count=0):
    f=1
    
    lec_time = dt.time(attend_time_hrs, attend_time_mins)
    curr_time = dt.datetime.now().time()
    while lec_time>curr_time:
        print("time left for ",course_name,' ',my_courses[course_name],' : ',int(time_diff(lec_time,curr_time).seconds/3600),"hour(s) ",int((time_diff(lec_time,curr_time).seconds/60)%60)," minutes ",
              time_diff(lec_time,curr_time).seconds%60," seconds")
        time.sleep(1)
        curr_time = dt.datetime.now().time()
        clear_output(wait=True)
    
    click_button(teams_button,try_this=10)
    time.sleep(1)
    p=click_button(search_bars,try_this=1000000)
    if(p==0):
        return
    time.sleep(1)
    keyboard.write(course_name)
    time.sleep(1)
    click(p.x,p.y+120)
    time.sleep(1)
    f=click_button(join_button,try_this=100)
    time.sleep(1)
    if f==0:
        if count<3:
            print('Request FAILED, trying again in 2 mins')
            time.sleep(120)
            curr_time = dt.datetime.now().time()
            if finish_time<curr_time:
                return
            attend_lec(course_name,attend_time_hrs,attend_time_mins,finish_time,count+1)
        return
    click_button(video_off,try_this=4,gs=False)
    click_button(select_audio,try_this=2)
    click_button(audio_off,try_this=3,gs=False)
    time.sleep(1)
    click_button(join_now)
    curr_time = dt.datetime.now().time()
    while finish_time>curr_time:
        print("In lecture ",course_name,' ',my_courses[course_name],'Time Left: ',int(time_diff(finish_time,curr_time).seconds/3600)," hour(s) ",int((time_diff(finish_time,curr_time).seconds/60)%60)," minutes ",
              time_diff(finish_time,curr_time).seconds%60," seconds")
        time.sleep(1)
        curr_time = dt.datetime.now().time()
        clear_output(wait=True)
    test=click_button(leave_call,try_this=5)
    if test==0:
        leave_call_func()

The course_timetable Dictionary should be filled corresponding to user's time table.

'week day (mon-sun)':\['\<lecture name or number\>,\<time(hrs:mins)\> \<duration in mins(eg: 60 for 1 hr class)\>','course_2,14:05 60'\]

In [8]:
week_day={0:'mon',1:'tue',2:'wed',3:'thu',4:'fri',5:'sat',6:'sun'}
course_timetable = {'mon':['EE626,12:00,60','EE672,14:00,60','RFIC,16:01,60'],
                    'tue':['EE442,11:00,60','EE626,12:00,60','EE672,14:00,60','EE 661,15:00,60','RFIC,16:01,60'],
                    'wed':['HS-401,12:00,60','EE672,14:00,60','EE 661,15:02,60'],
                    'thu':['HS-401,12:00,60','EE 661,15:00,60'],
                    'fri':['EE626,11:00,60','HS-401,12:00,60','RFIC,15:00,60']}
#course number should be present in the teams group name

In [9]:
x = dt.datetime.today().weekday()
for course in course_timetable[week_day[x]]:

    course_and_time=course.split(',',2)
    course_name=course_and_time[0]
    Time=course_and_time[1].split(':',1)

    attend_time_hrs = int(Time[0])
    attend_time_mins = int(Time[1])
    lecture_duration = int(course_and_time[2])
    
    finish_time=dt.timedelta(hours=attend_time_hrs, minutes=attend_time_mins) + dt.timedelta(hours=lecture_duration/60, minutes=lecture_duration%60)
    curr_time = dt.datetime.now().time()
    finish_time = (dt.datetime.min + finish_time).time()
    
    if finish_time<curr_time:
        print(course_name,':',my_courses[course_name],'is done ')
        continue
    else:
        print('Joining lecture ',course_name,':',my_courses[course_name],'till',finish_time)
        attend_lec(course_name,attend_time_hrs,attend_time_mins,finish_time)

print('Done For Today Now Enjoy!')

time left for  RFIC   RF circuits: Mahima Arawati  :  0 hour(s)  57  minutes  40  seconds


KeyboardInterrupt: 